In [ ]:
import PacTimeOrig.DataHandling as DH
import PacTimeOrig.Methods.utils as pacutils
from PacTimeOrig.Methods.Clusteresque import recusiveCDF, EMDRecursive, getvarforclustering, loadEMDmatrices

import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.cluster import AffinityPropagation

##

In [ ]:
#set some analysis variables that'll stay fixed
data = DH.dataloader(direc='C:\\Users\\Justin\\Documents\\GitHub\\PacManHuman\\', subject='H', session=1,
                     suffix='Pac_dACC.mat')
sessionVars = DH.ExperimentVarsRetrieve(data)
positions = DH.retrievepositions(data)
#velocity, accelaration
positions = pacutils.computederivatives(positions)
#distance and angle
positions = pacutils.computeheading(positions)
pacutils.computevelocitytowardstarget(positions)

#Append the above info into SessionVars as a goodbad (good =1), get rection time, and idx of first peak for splitting
sessionVars = pacutils.trialRtPkRej(sessionVars, positions)

In [ ]:
#Get aggregated features for each trial
p1trialdat=pacutils.getclusterfeatures(sessionVars, positions,winlength=1)

#Make sure not to include bad trials using the gb flag in sessionVars, let's make a new variable for it
p1trials = sessionVars[sessionVars['numNPC'] == 1].index
goodbad=sessionVars['goodtrial'].loc[p1trials].reset_index(drop=True)

In [ ]:
datum=[]

for i in p1trials:
    A=pd.DataFrame(positions[i]['selfXpos'].to_numpy())
    B=pd.DataFrame(positions[i]['selfXvel'].to_numpy())
    C=pd.DataFrame(positions[i]['prey1Xpos'].to_numpy())
    D=pd.DataFrame(positions[i]['prey1Xvel'].to_numpy())
    E=pd.DataFrame(positions[i]['prey1_scalar_projection'].to_numpy())
    F=pd.DataFrame(positions[i]['prey1_scalar_projection_gradient'].to_numpy())

    'prey1_scalar_projection'
    #obsA=pd.concat((A-C,B,D,E,F),axis=1).to_numpy()
    obsA=pd.concat((A-C,B,D,E,F),axis=1).loc[sessionVars['ReactionTime'][i]:,:].to_numpy()

    datum.append(obsA)

n_inputs=0
LLS=[]
for i in tqdm(range(10)):
    hmm = ssm.HMM(i+1, 5, observations="ar")
    lls=hmm.fit(datum,method="em")
    LLS.append(lls)

lla=[]
for i in (range(10)):
    lla.append(np.min(np.abs(LLS[i])))

#Find the best fitting model based on final likeihood min(np.abs(LLS))
hmm = ssm.HMM(5, 5,M=n_inputs, observations="ar")
lls=hmm.fit(datum,method="em")



In [ ]:
#Now get expected states through decoding and create some distributions for plotting
states=pd.DataFrame()
dat=pd.DataFrame()
for i in range(len(datum)):
    states=pd.concat((states,pd.DataFrame(hmm.most_likely_states(datum[i]))),axis=0)
    dat=pd.concat((dat,pd.DataFrame(datum[i])),axis=0)

states.reset_index(drop=True,inplace=True)
dat.reset_index(drop=True,inplace=True)

